## Часть 1
Напишите скрипт, который скачивает все данные прошедших губернаторских выборов для всех избирательных участков. 

Входная точка [по ссылке](http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222). Затем нужно перейти на сайты территориальных избирательных комиссий. Результаты нужно сохранить в  `cvs`-файл, `sqlite` базе данных или `parquet`-файле. В итоге должна получиться таблица с полями:
- название ТИК (территориальная избирательная комиссия, некоторое объединение избирательных участков на определенной территории)
- номер УИК (избирательный участок)
- 14 стандартных полей из итогового протокола

Скрипт должен быть в отдельном `*.py` файле или в отдельном `jupyter`-ноутбуке. 

In [1]:
import requests
import re
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import json
import numpy as np

In [2]:
ref = "http://www.st-petersburg.vybory.izbirkom.ru/region/region/st-petersburg?action=show&root=1&tvd=27820001217417&vrn=27820001217413&region=78&global=&sub_region=78&prver=0&pronetvd=null&vibid=27820001217417&type=222"
response = requests.get(ref)

In [3]:
soup = BeautifulSoup(response.text)
connection = sqlite3.connect("VoteStatistic.db")

In [4]:
table = soup.find('table',{ 'style' : 'width:100%;border-color:#000000' })
links=table.findAll('a', attrs={'href': re.compile("^http://")})

In [31]:
def preprocessCandidate(values):
    stats = []
    for x in values:
        stat = x.split(' ')
        stat[0] = int(stat[0])
        stat[1] = float(stat[1].strip('%'))
        stats.append(stat)
    return stats

def preprocessTable(table):
    for i in range(2,13):
        table[list(table)[i]] = table[list(table)[i]].astype(int)
        
    for i in range (13, 19, 2):
        values = table[table.columns[i]].values
        processedValues = np.array(preprocessCandidate(values))
        table[table.columns[i]] = processedValues[:, 0]
        table.insert(loc=i+1, column=table.columns[i] + " (%)", value=processedValues[:, 1])
    return table

In [32]:
for tic in links:
    table = BeautifulSoup(requests.get(tic.get("href")).text).find('table', {'style' : 'width:100%;border-color:#000000'}) 
    if table is not None:
        df = pd.read_html(str(table.findAll('td')), header=0)
        resultTable = df[0].dropna().join(df[1].dropna()).T
        resultTable.rename(index={"Unnamed: 1": "Статистика по"}, inplace=True)
        resultTable.reset_index(inplace=True)
        newCol=[tic.text for i in range(resultTable.shape[0])]
        newCol[1] = "ТИК"
        resultTable.insert(loc=0, column='ТИК', value=newCol)
        resultTable.columns = resultTable.iloc[1]
        resultTable = resultTable[2:]
        preprocessTable(resultTable).to_sql("tableStatistic", connection, if_exists="append", index=False)

In [33]:
pd.read_sql("SELECT * from tableStatistic", connection)

,ТИК,Статистика по,"Число избирателей, внесенных в список избирателей на момент окончания голосования","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных избирателям, проголосовавшим вне помещения для голосования",Число погашенных избирательных бюллетеней,"Число избирательных бюллетеней, содержащихся в переносных ящиках для голосования","Число избирательных бюллетеней, содержащихся в стационарных ящиках для голосования",Число недействительных избирательных бюллетеней,Число действительных избирательных бюллетеней,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Амосов Михаил Иванович,Амосов Михаил Иванович (%),Беглов Александр Дмитриевич,Беглов Александр Дмитриевич (%),Тихонова Надежда Геннадьевна,Тихонова Надежда Геннадьевна (%)
0,Территориальная избирательная комиссия №1,Сумма,105510,83250,31626,1592,50032,1590,31437,1220,31807,0,0,6206.0,18.79,19818.0,60.01,5783.0,17.51
1,Территориальная избирательная комиссия №1,УИК №1,1803,1200,587,11,602,11,587,23,575,0,0,110.0,18.39,345.0,57.69,120.0,20.07
2,Территориальная избирательная комиссия №1,УИК №2,1466,1100,433,14,653,14,430,19,425,0,0,53.0,11.94,326.0,73.42,46.0,10.36
3,Территориальная избирательная комиссия №1,УИК №3,2092,1600,576,22,1002,22,576,15,583,0,0,155.0,25.92,332.0,55.52,96.0,16.05
4,Территориальная избирательная комиссия №1,УИК №4,1056,1000,318,4,678,4,318,13,309,0,0,67.0,20.81,171.0,53.11,71.0,22.05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4097,Территориальная избирательная комиссия №30,УИК №2350,284,300,166,53,81,53,166,0,219,0,0,21.0,9.59,184.0,84.02,14.0,6.39
4098,Территориальная избирательная комиссия №30,УИК №2351,132,150,86,27,37,27,86,2,111,0,0,7.0,6.19,91.0,80.53,13.0,11.50
4099,Территориальная избирательная комиссия №30,УИК №2352,320,350,217,23,110,23,217,4,236,0,0,20.0,8.33,193.0,80.42,23.0,9.58
4100,Территориальная избирательная комиссия №30,УИК №2355,485,500,335,12,153,12,335,2,345,0,0,23.0,6.63,290.0,83.57,32.0,9.22
